In [14]:
# fs, lang = 22050, "English"
# corpus = "ljspeech"
# speech_model = "conformer_fastspeech2"
# # speech_model = "fastspeech2"
# # speech_model = "tacotron2"
# tag = "kan-bayashi/" + corpus + "_" + speech_model
# # tag = "kan-bayashi/ljspeech_tacotron2"
# # tag = "kan-bayashi/jsut_tacotron2"
# # tag = "kan-bayashi/ljspeech_fastspeech"
# # tag = "kan-bayashi/ljspeech_fastspeech2"
# # tag = "kan-bayashi/ljspeech_conformer_fastspeech2"
# vocoder_tag = "ljspeech_parallel_wavegan.v3"

fs, lang = 24000, "Mandarin"
# tag = "kan-bayashi/csmsc_tacotron2"
# tag = "kan-bayashi/csmsc_transformer"
# tag = "kan-bayashi/csmsc_fastspeech"
# tag = "kan-bayashi/csmsc_fastspeech2"
tag = "kan-bayashi/csmsc_conformer_fastspeech2"
vocoder_tag = "csmsc_parallel_wavegan.v1"
# vocoder_tag = "csmsc_multi_band_melgan.v2"

import time
import torch
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.tts_inference import Text2Speech
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model
d = ModelDownloader()
text2speech = Text2Speech(
    **d.download_and_unpack(tag),
    device="cuda",
    # Only for Tacotron 2
    threshold=0.5,
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2
    speed_control_alpha=1.0,
)
# text2speech.spc2wav = None  # Disable griffin-lim
# NOTE: Sometimes download is failed due to "Permission denied". That is
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag, download_dir='./models')).to("cuda").eval()
vocoder.remove_weight_norm()

In [15]:
t = "北京市"
t = "北京市"
wav = None
with torch.no_grad():
    start = time.time()
    wav, c, *_ = text2speech(t)
    wav = vocoder.inference(c)
rtf = (len(wav) / fs) / (time.time() - start)
print(f"Speed: {rtf:5f}x")

from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=fs))

KeyboardInterrupt: 

In [3]:
d